In [56]:
import argparse
import pickle
from pathlib import Path

import fraud_eagle as feagle
import fraudar
import networkx as nx
import numpy as np
import pandas as pd
import rsd

from rev2 import rev2compute
from utils import load_data, split_data_by_time

import seaborn as sns
import matplotlib.pyplot as plt

import sklearn.metrics

In [47]:
def get_results(data_name="alpha", alg="fraudar", attack_name="ilp_attack", budget=100, frac=0.2):
    data_nw_df, data_gt_df = load_data(data_name=data_name)
    p = Path(f"../res/{attack_name}/{alg}-{data_name}/{budget}-{frac}.pkl")
    if not p.exists():
        print(p)
        return None
    with open(p, "rb") as fp:
        scores = pickle.load(fp)
        scores = dict(zip(range(len(scores)), scores))
        # 1 means fraudster, 0 means benign
        scores["truth"] = dict(zip(data_gt_df["id"], (1-data_gt_df["label"])/2))
    # df = pd.DataFrame({sp: {u: scores[sp][u] for u in scores[sp] if u in data_gt_df["id"].tolist()} for sp in scores})
    df = pd.DataFrame({sp: {u: scores[sp][u] for u in scores[sp]} for sp in scores})
    df.loc[[u[:5]=="usock" for u in df.index.tolist()], "truth"] = 1
    df.loc[[u[:6]=="udummy" for u in df.index.tolist()], "truth"] = 0
    return df

In [48]:
def get_metrics(df_res, splits=4, K=None):
    if df_res is None:
        return None
    ret = {}
    for sp in range(splits):
        # df_res["truth"] = df_res["truth"].fillna(1)
#         df_res.loc[[u[:5]=="usock" for u in df_res.index.tolist()], "truth"] = 1
#         df_res.loc[[u[:6]=="udummy" for u in df_res.index.tolist()], "truth"] = 0
        df = df_res[[sp, "truth"]].dropna()
        t = df["truth"].sum() if K is None else K
        t_frac = t / df.shape[0]
        # df["pred"] = (df[sp].rank(ascending=False, method="min") <= t) * 1
        # df["pred"] = df[sp] >= t_frac
        df["pred"] = df[sp] >= 0.5

        ret[sp] = {
            "precision": sklearn.metrics.precision_score(y_true=df["truth"], y_pred=df["pred"]),
            "recall": sklearn.metrics.recall_score(y_true=df["truth"], y_pred=df["pred"]),
            "f1": sklearn.metrics.f1_score(y_true=df["truth"], y_pred=df["pred"])
        }
    return ret

In [64]:
alg_list = ["fraudar", "rsd", "rev2"][:2]
data_list = ["alpha", "otc", "amazon", "epinions"][:2]
budget_list = [100.0, 200.0, 300.0, 400.0]
frac_list = [0.2, 0.4, 0.6, 0.8]

In [65]:
attack_res = {
    (d, a, b, f): get_results(data_name=d, alg=a, attack_name="ilp_attack", budget=b, frac=f)
    for d in data_list for a in alg_list
    for b in budget_list for f in frac_list
}
dict_met = {
    (d, a, b, f): pd.DataFrame.from_dict(get_metrics(df_res=attack_res[(d, a, b, f)], splits=4, K=None))
    for d, a, b, f in attack_res
#     for b in budget_list for f in frac_list
}
df_met = pd.concat(dict_met, keys=dict_met.keys())
display(df_met)

../res/ilp_attack/fraudar-otc/100.0-0.2.pkl
../res/ilp_attack/fraudar-otc/100.0-0.4.pkl
../res/ilp_attack/fraudar-otc/100.0-0.6.pkl
../res/ilp_attack/fraudar-otc/100.0-0.8.pkl
../res/ilp_attack/fraudar-otc/200.0-0.2.pkl
../res/ilp_attack/fraudar-otc/200.0-0.4.pkl
../res/ilp_attack/fraudar-otc/200.0-0.6.pkl
../res/ilp_attack/fraudar-otc/200.0-0.8.pkl
../res/ilp_attack/fraudar-otc/300.0-0.2.pkl
../res/ilp_attack/fraudar-otc/300.0-0.4.pkl
../res/ilp_attack/fraudar-otc/300.0-0.6.pkl
../res/ilp_attack/fraudar-otc/300.0-0.8.pkl
../res/ilp_attack/fraudar-otc/400.0-0.2.pkl
../res/ilp_attack/fraudar-otc/400.0-0.4.pkl
../res/ilp_attack/fraudar-otc/400.0-0.6.pkl
../res/ilp_attack/fraudar-otc/400.0-0.8.pkl
../res/ilp_attack/rsd-otc/100.0-0.2.pkl
../res/ilp_attack/rsd-otc/100.0-0.4.pkl
../res/ilp_attack/rsd-otc/100.0-0.6.pkl
../res/ilp_attack/rsd-otc/100.0-0.8.pkl
../res/ilp_attack/rsd-otc/200.0-0.2.pkl
../res/ilp_attack/rsd-otc/200.0-0.4.pkl
../res/ilp_attack/rsd-otc/200.0-0.6.pkl
../res/ilp_attac

0         1         2         3
alpha fraudar 100.0 0.2 precision  0.350785  0.370968  0.380682  0.335526
                        recall     0.837500  0.811765  0.848101  0.698630
                        f1         0.494465  0.509225  0.525490  0.453333
                    0.4 precision  0.364103  0.364641  0.380682  0.364198
                        recall     0.845238  0.741573  0.807229  0.766234
...                                     ...       ...       ...       ...
      rsd     400.0 0.6 recall     0.645161  0.658915  0.691057  0.769231
                        f1         0.504732  0.504451  0.518293  0.543807
                    0.8 precision  0.396694  0.398467  0.402390  0.402299
                        recall     0.685714  0.717241  0.726619  0.789474
                        f1         0.502618  0.512315  0.517949  0.532995

[96 rows x 4 columns]

In [58]:
attack_res[("alpha", "fraudar", 400.0, 0.2)].to_csv("al.csv")

In [59]:
df_met.index = df_met.index.set_names(["data", "algo", "budget", "frac", "metrics"])

In [60]:
pd.concat([df_met.loc[("alpha", "fraudar", slice(None), slice(None))], df_met.loc[("alpha", "fraudar", slice(None), slice(None))].mean(axis=1)], axis=1)

0         1         2         3         0
budget frac metrics                                                    
100.0  0.2  precision  0.350785  0.370968  0.380682  0.335526  0.359490
            recall     0.837500  0.811765  0.848101  0.698630  0.798999
            f1         0.494465  0.509225  0.525490  0.453333  0.495628
       0.4  precision  0.364103  0.364641  0.380682  0.364198  0.368406
            recall     0.845238  0.741573  0.807229  0.766234  0.790068
            f1         0.508961  0.488889  0.517375  0.493724  0.502237
       0.6  precision  0.364103  0.370968  0.380682  0.364198  0.369987
            recall     0.806818  0.741935  0.770115  0.728395  0.761816
            f1         0.501767  0.494624  0.509506  0.485597  0.497873
       0.8  precision  0.357513  0.370968  0.380682  0.364198  0.368340
            recall     0.750000  0.711340  0.736264  0.694118  0.722930
            f1         0.484211  0.487633  0.501873  0.477733  0.487862
200.0  0.2  precision  0.357513  0.377660  0.380682  0.364198  0.370013
            recall     0.821429  0.797753  0.807229  0.766234  0.798161
            f1         0.498195  0.512635  0.517375  0.493724  0.505482
       0.4  precision  0.357513  0.382199  0.380682  0.335526  0.363980
            recall     0.750000  0.752577  0.736264  0.600000  0.709710
            f1         0.484211  0.506944  0.501873  0.430380  0.480852
       0.6  precision  0.357513  0.377660  0.380682  0.335526  0.362845
            recall     0.690000  0.676190  0.676768  0.548387  0.647836
            f1         0.470990  0.484642  0.487273  0.416327  0.464808
       0.8  precision  0.357513  0.380208  0.380682  0.364198  0.370650
            recall     0.638889  0.646018  0.626168  0.584158  0.623808
            f1         0.458472  0.478689  0.473498  0.448669  0.464832
300.0  0.2  precision  0.357513  0.398990  0.384181  0.364198  0.376220
            recall     0.784091  0.849462  0.781609  0.728395  0.785889
            f1         0.491103  0.542955  0.515152  0.485597  0.508702
       0.4  precision  0.357513  0.386598  0.404372  0.364198  0.378170
            recall     0.690000  0.714286  0.747475  0.634409  0.696542
            f1         0.470990  0.501672  0.524823  0.462745  0.490057
       0.6  precision  0.357513  0.389744  0.473430  0.364198  0.396221
            recall     0.616071  0.649573  0.882883  0.561905  0.677608
            f1         0.452459  0.487179  0.616352  0.441948  0.499485
       0.8  precision  0.364103  0.511905  0.380682  0.364198  0.405222
            recall     0.572581  1.000000  0.544715  0.504274  0.655392
            f1         0.445141  0.677165  0.448161  0.422939  0.498352
400.0  0.2  precision  0.357513  0.440909  0.380682  0.335526  0.378658
            recall     0.750000  1.000000  0.736264  0.600000  0.771566
            f1         0.484211  0.611987  0.501873  0.430380  0.507113
       0.4  precision  0.357513  0.383420  0.380682  0.356250  0.369466
            recall     0.638889  0.654867  0.626168  0.564356  0.621070
            f1         0.458472  0.483660  0.473498  0.436782  0.463103
       0.6  precision  0.357513  0.380208  0.384181  0.364198  0.371525
            recall     0.556452  0.565891  0.552846  0.504274  0.544866
            f1         0.435331  0.454829  0.453333  0.422939  0.441608
       0.8  precision  0.357513  0.355556  0.384181  0.356250  0.363375
            recall     0.492857  0.441379  0.489209  0.428571  0.463004
            f1         0.414414  0.393846  0.430380  0.389078  0.406930

In [63]:
df_met.mean(axis=1).to_frame().reset_index().pivot(index=["data", "algo", "frac", "metrics"], columns=["budget"]).loc[("otc", "rsd", slice(None), "f1")].droplevel(0, axis=1)

KeyError: 'otc'